In [1]:
%pip install pyspark
%pip install pymongo
%pip install dnspython

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark
import pymongo
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame


In [5]:
%pip install mongo-spark-connector

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/5c/f2/f3faa20684729d3910af2ee142e30432c7a46a817eadeeab87366ed87bbb/charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl.metadata
  Using cached charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl.metadata (31 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("JupyterMongoIntegration") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/JTXBigDataCluster.ElectricityLoadPrototype") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/JTXBigDataCluster.ElectricityLoadPrototype") \
    .getOrCreate()


In [8]:
# Read the CSV file into a DataFrame
file_path = "./data/continuous dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(10, truncate=False)


+-------------------+----------+------------------+-----------+------------+------------------+------------------+-----------+------------+------------------+------------------+-----------+-----------+------------------+----------+-------+------+
|datetime           |nat_demand|T2M_toc           |QV2M_toc   |TQL_toc     |W2M_toc           |T2M_san           |QV2M_san   |TQL_san     |W2M_san           |T2M_dav           |QV2M_dav   |TQL_dav    |W2M_dav           |Holiday_ID|holiday|school|
+-------------------+----------+------------------+-----------+------------+------------------+------------------+-----------+------------+------------------+------------------+-----------+-----------+------------------+----------+-------+------+
|2015-01-03 01:00:00|970.345   |25.865258789062523|0.018576382|0.016174316 |21.85054581787518 |23.482446289062523|0.017271755|0.0018553734|10.328948729384228|22.662133789062523|0.016562222|0.09609985 |5.364147952093894 |0         |0      |0     |
|2015-01-03 

In [10]:
# Write DataFrame to a JSON file
df.write.json("./data/continuous_dataset.json")



Py4JJavaError: An error occurred while calling o61.json.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

In [ ]:
read_df = spark.read.format("mongo").load()
read_df.show()
